<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/EnsembleLearning_RandomForest_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import datasets
import numpy as np


In [37]:
from sklearn.model_selection import train_test_split
iris = datasets.make_moons(n_samples=1000, noise=0.4)
X = iris[0]
y = iris[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [40]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

#3개의 분류기의 예측 클래스중 비중이 높은 클래스를 선택한다.
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

In [41]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.82
RandomForestClassifier 0.84
SVC 0.845
VotingClassifier 0.8375


In [46]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

#결정 트리 500개를 이용해서, 무작위 100개의 sample을 추출한 후 훈련한다.
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

VotingClassifier 0.845


In [48]:
#oob 평가
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
#훈련이 끝난시점 자동으로 oob 평가를 수행한다.
print(clf.__class__.__name__, bag_clf.oob_score_)

VotingClassifier 0.87
